##Análisis de VaR para Tickers del IPSA con Diferentes Niveles de Volumen

En este análisis, se calcularon diferentes estimaciones de Valor en Riesgo (VaR) para los principales tickers del Índice de Precios Selectivo de Acciones (IPSA), agrupándolos según sus niveles de volumen transaccional. Se seleccionaron los cinco tickers con mayor volumen y los cinco con menor volumen, evaluando su riesgo a través de diversas metodologías: VaR Histórico, VaR por Covarianza (o paramétrico), VaR mediante Simulación de Monte Carlo, y VaR ajustado por Cornish-Fisher. Todos los cálculos se realizaron considerando un nivel de confianza del 95%.

Posibles Descubrimientos

El análisis de VaR permite identificar diferencias clave en el perfil de riesgo entre los activos más líquidos (mayor volumen) y los menos líquidos (menor volumen):

 * Diferencias en el Riesgo Relativo: Los activos con menor volumen suelen presentar mayor volatilidad, lo que se refleja en valores de VaR más altos, especialmente en metodologías que capturan colas gruesas, como el método Cornish-Fisher o Monte Carlo.
 * Impacto de la Liquidez: Los activos con mayor volumen pueden mostrar menor sensibilidad al riesgo extremo debido a la mayor estabilidad en sus precios.
 * Ajustes por Asimetría y Curtosis: Las metodologías avanzadas, como Cornish-Fisher, permiten capturar mejor las características de activos con distribuciones no normales, mostrando valores ajustados que pueden diferir significativamente de los modelos tradicionales.

Utilidad del Análisis

Este análisis es crucial para:

 * Gestión de Riesgos: Identificar los activos más propensos a pérdidas extremas, ayudando a construir portafolios más resilientes.
 * Toma de Decisiones: Proporcionar información para ajustar estrategias de inversión según el perfil de riesgo de cada activo.
 * Comprensión de la Dinámica del Mercado: Evaluar cómo el volumen de transacciones influye en la estabilidad y el riesgo de los activos del IPSA.

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import norm
from scipy.stats import skew, kurtosis, skewtest
from scipy.stats.mstats import gmean
from scipy.stats import jarque_bera

In [2]:
#obtener lista de tickers
# Wikipedia URL for IPSA components
url = 'https://es.wikipedia.org/wiki/%C3%8Dndice_de_Precio_Selectivo_de_Acciones'

# Extract tables from the Wikipedia page
tables = pd.read_html(url)

# Display the first table (if it exists)
print(tables[0])

    N.º   Nemotécnico                             Empresa
0     1       AGUAS-A                       Aguas Andinas
1     2      ANDINA-B                Embotelladora Andina
2     3           BCI      Banco de Crédito e Inversiones
3     4    BSANTANDER               Banco Santander Chile
4     5           CAP                                 CAP
5     6           CCU      Compañía de Cervecerías Unidas
6     7    CENCOSHOPP              Cencosud Shopping S.A.
7     8      CENCOSUD                       Cencosud S.A.
8     9         CHILE                      Banco de Chile
9    10          CMPC                       Empresas CMPC
10   11        COLBUN                              Colbún
11   12    CONCHATORO                  Viña Concha y Toro
12   13         COPEC                               Copec
13   14           ECL                               Engie
14   15        ENELAM                       Enel Américas
15   16     ENELCHILE                          Enel Chile
16   17       

In [3]:
df_idx=[i + '.SN' for i in tables[0]['Nemotécnico']]
tables[0]['Nemotécnico']=[i + '.SN' for i in tables[0]['Nemotécnico']]

In [4]:
#Se validará el funcionamiento de los tickers proporcionados, identificando y filtrando aquellos que no retornan información válida.

def validate_tickers(ticker_list):
    """
    Validate a list of tickers to check which ones are valid in yfinance.
    Parameters:
        ticker_list (list): List of tickers to validate.
    Returns:
        valid_tickers (list): A list of valid tickers.
        invalid_tickers (list): A list of invalid tickers.
    """
    valid_tickers = []
    invalid_tickers = []

    for ticker in ticker_list:
        try:
            # Attempt to fetch basic info for the ticker
            stock = yf.Ticker(ticker)

            # Try fetching a common piece of information like "info" or historical data
            if stock.info.get("bookValue") is None:
                print(f"Ticker {ticker} is invalid (no data in info).")
                invalid_tickers.append(ticker)
            else:
                #print(f"Ticker {ticker} is valid.")
                valid_tickers.append(ticker)
        except Exception as e:
            # Handle specific errors like 404 Client Errors
            print(f"An error occurred with ticker {ticker}: {e}")
            invalid_tickers.append(ticker)

    return valid_tickers, invalid_tickers


In [5]:
#ticker list
ticker_list = df_idx

# Validate tickers
valid_tickers, invalid_tickers = validate_tickers(ticker_list)

print("\nValid Tickers:", valid_tickers)
print("Invalid Tickers:", invalid_tickers)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CENCOSHOPP.SN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CENCOSHOPP.SN&crumb=nV3KShCxafg


Ticker CENCOSHOPP.SN is invalid (no data in info).
Ticker ORO%20BLANCO.SN is invalid (no data in info).

Valid Tickers: ['AGUAS-A.SN', 'ANDINA-B.SN', 'BCI.SN', 'BSANTANDER.SN', 'CAP.SN', 'CCU.SN', 'CENCOSUD.SN', 'CHILE.SN', 'CMPC.SN', 'COLBUN.SN', 'CONCHATORO.SN', 'COPEC.SN', 'ECL.SN', 'ENELAM.SN', 'ENELCHILE.SN', 'ENTEL.SN', 'FALABELLA.SN', 'IAM.SN', 'ITAUCL.SN', 'LTM.SN', 'MALLPLAZA.SN', 'PARAUCO.SN', 'QUINENCO.SN', 'RIPLEY.SN', 'SMU.SN', 'SONDA.SN', 'SQM-B.SN', 'VAPORES.SN']
Invalid Tickers: ['CENCOSHOPP.SN', 'ORO%20BLANCO.SN']


Obtencion de los ticker con mayor y menor volumen del año 2023 del IPSA

In [32]:
# Elegir Fecha periodo de muestra
start_date = '2023-01-01'
end_date = '2024-01-01'

data = yf.download(valid_tickers, start=start_date, end=end_date)['Volume']

[*********************100%***********************]  28 of 28 completed


In [33]:
data.head()

Ticker,AGUAS-A.SN,ANDINA-B.SN,BCI.SN,BSANTANDER.SN,CAP.SN,CCU.SN,CENCOSUD.SN,CHILE.SN,CMPC.SN,COLBUN.SN,...,ITAUCL.SN,LTM.SN,MALLPLAZA.SN,PARAUCO.SN,QUINENCO.SN,RIPLEY.SN,SMU.SN,SONDA.SN,SQM-B.SN,VAPORES.SN
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,116573,33544,7825,9392043,4907,8456,285455,17680832,255961,843765,...,107116,29737207,968,88851,5909,203,114572,234729,32978,1541025
2023-01-04,163244,77937,10697,14840191,1790,32665,100000,8971749,235359,380520,...,123128,8191972,87164,162557,8499,28953,1287013,1409,26088,4942775
2023-01-05,701504,289362,65568,8638953,168240,57733,1131311,106588952,1690435,6991788,...,95111,200405353,102927,616435,62855,762353,866351,313771,185029,10413117
2023-01-06,39981,35358,4643,1688317,13879,3093,160420,2606585,209448,579914,...,332671,83000,8000,7937,17757,4901,447593,13533,19984,2212339
2023-01-09,58469,37005,45890,10631460,15856,18915,268776,8130084,256723,1000938,...,1034441,1066418,10949,143872,10583,11804,279966,232573,22920,3754232


In [34]:
# Agrupar por ticker y sumar el volumen
bottom_5=data.mean().sort_values().reset_index().head(5)
top_5=data.mean().sort_values().reset_index().tail(5)

#revisaremos el VaR de las 5 mas transadas y las 5 menos transadas
ticker_list=bottom_5['Ticker'].tolist()+top_5['Ticker'].tolist()

In [35]:
data_close = yf.download(ticker_list,start=start_date,end=end_date)['Adj Close']

[*********************100%***********************]  10 of 10 completed


In [36]:
returns = data_close.pct_change()
returns = returns.dropna()

In [37]:
df_1=pd.DataFrame(columns=['Ticker', 'Semi_desviacion', 'Des_Estandar'])

El Valor en Riesgo (VaR) es una técnica financiera que se utiliza para medir el riesgo de una inversión, cartera o empresa en un periodo de tiempo determinado\
Se define como 'la máxima pérdida esperada en un periodo de tiempo y con un nivel de confianza dados, encondiciones normales de mercado'. \
Se procedera a calcular de diferentes formas el VaR

In [38]:
for i,ticker in enumerate(returns.columns):
    # Calcular la media de los retornos
  mean_return = returns[ticker].mean()
  std_return =  returns[ticker].std()
    # Filtrar solo los retornos negativos (desviaciones negativas)
  negative_deviations = returns[ticker][returns[ticker] < mean_return]
    # Calcular la semidesviación
  semi_deviation = np.sqrt(np.sum((negative_deviations - mean_return) ** 2) / len(returns))
  df_1.loc[i,['Ticker', 'Semi_desviacion', 'Des_Estandar']]=ticker, semi_deviation, std_return

Var método de simulación Histórica.\
Es un método aplicable tanto a carteras
lineales como no lineales, debido a que es un método no paramétrico, que
no depende de ninguna de las hipótesis sobre distribuciones de probabilidad
subyacente y, por lo tanto, permite capturar el fenómeno de las colas
gruesas al mismo tiempo que elimina la necesidad de estimar y trabajar con
volatilidades y correlaciones, evitando en gran medida el riesgo de
modelización.\
Ventajas: Las ventajas fueron mencionadas en su definición, y hace que
este método sea preferible al de matriz varianza-covarianza, especialmente
cuando se trata del cálculo del VaR para carteras de instrumentos no
lineales.\
Desventajas: La principal desventaja del método de simulación histórica
viene dada por las características de los datos utilizados, que supone que
ningún evento que no haya ocurrido en el pasado podrá ocurrir en el futuro.

In [39]:
niveles_confianza = 0.95  # Niveles de confianza: 95% y 99%
for i,ticker in enumerate(returns.columns):
    var_historico = np.percentile(returns[ticker], (1 - niveles_confianza) * 100)
    df_1.loc[i,['VaRH_95']]=var_historico

VaR método de varianza-covarianza.\
Este método se basa en la hipótesis de la distribución conjunta normal de los rendimientos de la cartera, y en la hipótesis de la relación lineal (a lo más cuadrática) entre los factores de riesgo del mercado y el valor de la empresa.\
Ventajas: La estimación de VaR a través de éste método es simple y rápida,
lo que lo hace atractivo cuando se trabaja en tiempo real.\
Desventajas: Sobreestima el VaR para niveles bajos de confianza, y lo
subestima para niveles altos de confianza (inconveniente derivado de la
suposición de normalidad en los retornos de la cartera y no capturar el
fenómeno de las colas gruesas).

In [41]:
# VaR método de varianza-covarianza
for i,ticker in enumerate(returns.columns):
  mean_return = returns[ticker].mean()
  std_dev = returns[ticker].std()

  # Calcular el VaR con un nivel de confianza del 95 % utilizando la puntuación Z
  confidence_level = 0.95
  z_score = norm.ppf( 1 - confidence_level)
  VaR_varianza_covarianza = mean_return + z_score*std_dev
  df_1.loc[i,['Var_Cov']]=VaR_varianza_covarianza


VaR Monte Carlo.\
La Simulación de Monte Carlo es un método global, aplicable tanto de forma paramétrica como no paramétrica, que elimina la necesidad de aproximaciones que introducen imprecisiones. Es útil para valorar posiciones lineales y no lineales.

 * Modelo no paramétrico: No depende de una distribución de probabilidad específica, lo que reduce el riesgo de modelización y permite capturar fenómenos como colas gruesas sin depender de volatilidades y correlaciones.
 * Ventajas: Ofrece una visión más realista del riesgo, ya que refleja todas las posibles realizaciones de precios y sus probabilidades.
 * Desventajas: Su principal limitación es la lentitud computacional, aunque esto se está mitigando con los avances tecnológicos en informática.

In [15]:
#calculo VaR Monte-Carlo
for i,ticker in enumerate(returns.columns):
  # Simular retornos futuros usando Monte Carlo
  num_simulations = 10000
  simulation_horizon = 252   # Número de días de negociación en un año
  simulated_returns = np.random.normal(np.mean(returns[ticker]), np.std(returns[ticker]), (simulation_horizon, num_simulations))

  # Calcular los valores simulados de la cartera
  initial_investment = 1000000   # $1,000,000
  portfolio_values = initial_investment * np.exp(np.cumsum(simulated_returns, axis= 0 ))

  # Calcular los retornos de la cartera
  portfolio_returns = portfolio_values[- 1 ] / portfolio_values[ 0 ] - 1

  # Calcular el VaR al 95% de nivel de confianza
  confidence_level = 0.95
  VaR_monte_carlo = np.percentile(portfolio_returns, ( 1 - confidence_level) * 100 )
  df_1.loc[i,['Monte_Carlo']]=VaR_monte_carlo



VaR Cornish-Fisher

El método Cornish-Fisher ajusta el cálculo del VaR mediante la expansión de Cornish-Fisher, que permite incluir los efectos de asimetría (skewness) y curtosis (kurtosis) en la distribución de los rendimientos.

 * Características: Este enfoque adapta el VaR tradicional, que asume una distribución normal, para capturar mejor los riesgos en distribuciones no normales.
 * Ventajas: Es más preciso en escenarios con colas gruesas o distribuciones asimétricas y mantiene un cálculo eficiente, ya que no requiere simulaciones complejas.
 * Desventajas: La precisión depende de que las estimaciones de asimetría y curtosis sean fiables, lo que puede ser un desafío en muestras pequeñas o con datos inestables.

In [17]:
#VaR Cornish-Fisher
for i,ticker in enumerate(returns.columns):
  # Supongamos que tenemos los retornos diarios de un activo
  #np.random.seed(40)

  # Calcular parámetros básicos
  mean_return = np.mean(returns[ticker])
  std_return = np.std(returns[ticker])
  skewness = skew(returns[ticker])
  kurt = kurtosis(returns[ticker], fisher=True) + 3  # Curtosis total (no solo excesiva)

  # Nivel de confianza
  confidence_level = 0.05
  z_alpha = -1.645  # Para el 95% de confianza

  # Ajuste Cornish-Fisher
  z_adjusted = (
      z_alpha
      + (z_alpha**2 - 1) * skewness / 6
      + (z_alpha**3 - 3 * z_alpha) * (kurt - 3) / 24
      - (2 * z_alpha**3 - 5 * z_alpha) * (skewness**2) / 36
  )

  # Calcular VaR usando Cornish-Fisher
  var_cornish_fisher = -z_adjusted * std_return + mean_return
  df_1.loc[i,['Cornish_Fisher']]=var_cornish_fisher


In [18]:
df_1

,Ticker,Semi_desviacion,Des_Estandar,VaRH_95,Var_Cov,Monte_Carlo,Cornish_Fisher
0,BCI.SN,0.010161,0.015264,-0.022869,-0.023888,-0.082983,0.023112
1,BSANTANDER.SN,0.009714,0.013332,-0.023236,-0.020527,-0.000819,0.024356
2,CAP.SN,0.013555,0.019538,-0.032241,-0.031577,-0.308677,0.031888
3,CHILE.SN,0.00832,0.012425,-0.018301,-0.019141,0.009309,0.020009
4,COPEC.SN,0.010615,0.01577,-0.023590,-0.025657,-0.282409,0.023596
5,ENELCHILE.SN,0.013186,0.018463,-0.027329,-0.028439,0.005819,0.033064
6,ITAUCL.SN,0.007413,0.01244,-0.014956,-0.020307,-0.244689,0.005235
7,LTM.SN,0.015026,0.022768,-0.034064,-0.035186,-0.028051,0.034546
8,SQM-B.SN,0.021029,0.02901,-0.040364,-0.048193,-0.589803,0.049294
9,VAPORES.SN,0.020877,0.027081,-0.026692,-0.045038,-0.561287,0.038011


El valor en riesgo (VaR) es una métrica vital para evaluar el riesgo financiero, ya que ofrece una cuantificación clara de las pérdidas potenciales. Al implementar el VaR con Python, los inversores y los gestores de riesgos pueden obtener información útil sobre el perfil de riesgo de su cartera. Los ejemplos proporcionados con el método histórico, el método de varianza-covarianza y la simulación de Monte Carlo demuestran lo versátil y potente que puede ser el VaR en la gestión del riesgo financiero.

##Palabras Finales
  Volumen Bajo (Primeros 5 Tickers):\
        Los tickers con menor volumen de transacciones (BCI.SN, BSANTANDER.SN, CAP.SN, CHILE.SN, COPEC.SN) presentan:\
            Valores más bajos de semi-desviación y desviación estándar en comparación con los tickers de mayor volumen.\
            Por ejemplo, CHILE.SN tiene una semi-desviación de 0.0083 y desviación estándar de 0.0124, mucho menor que SQM-B.SN (0.0210 y 0.0290, respectivamente).\
        Esto sugiere que los tickers con menor volumen tienden a ser menos volátiles, lo que puede reflejar una menor liquidez y actividad de mercado que suaviza las fluctuaciones de precios.

  Volumen Alto (Últimos 5 Tickers):\
        Los tickers con mayor volumen (ENELCHILE.SN, ITAUCL.SN, LTM.SN, SQM-B.SN, VAPORES.SN) tienen:\
            Valores de semi-desviación y desviación estándar notablemente más altos.\
            Por ejemplo, VAPORES.SN tiene una semi-desviación de 0.0208 y desviación estándar de 0.0271, casi el doble que los tickers de menor volumen.\
        Esto indica que los activos con mayor volumen son más propensos a fluctuaciones significativas, probablemente debido a una mayor participación de inversores y reacciones más rápidas a eventos de mercado.

Análisis del VaR con Volumen

  VaR Histórico (VaRH_95):\
        Los tickers con mayor volumen (SQM-B.SN, VAPORES.SN) presentan los valores de VaR histórico más conservadores (−0.0404 y −0.0266), reflejando una mayor probabilidad de pérdidas extremas.
        En contraste, los tickers de menor volumen tienen VaR más moderados (−0.0183 para CHILE.SN), lo que indica un menor riesgo percibido.

  Cornish-Fisher:\
        Los ajustes de asimetría y curtosis son más marcados para los tickers de mayor volumen, lo que se traduce en valores más altos (0.0492 para SQM-B.SN, 0.0380 para VAPORES.SN). Esto refleja una mayor exposición a eventos de cola en activos con alta actividad.

Nivel de Confianza (95%)

  Todas las métricas de VaR se calcularon con un nivel de confianza del 95%, lo que significa que existe un 5% de probabilidad de que las pérdidas superen los valores estimados.\
  Esto subraya que los valores de VaR no representan pérdidas máximas absolutas, sino un umbral basado en datos históricos y/o supuestos estadísticos.\
  La elección del 95% de confianza puede ser adecuada para análisis de riesgo moderado, pero un nivel del 99% podría ser más útil para escenarios conservadores o activos de mayor volumen como SQM-B.SN y VAPORES.SN.\

Conclusión Adicional sobre el Volumen y el Riesgo

  Tickers de Volumen Bajo:\
        Presentan menor riesgo a la baja y menos volatilidad general, lo que los hace más atractivos para inversores con baja tolerancia al riesgo.

  Tickers de Volumen Alto:\
        Aunque presentan mayores riesgos y volatilidad, estos activos también podrían ofrecer mayores oportunidades de retorno, ya que su mayor volumen suele ser reflejo de alta liquidez y participación activa del mercado.

  Diversificación:\
        Dada la diferencia en los perfiles de riesgo entre los tickers de alto y bajo volumen, una estrategia diversificada podría aprovechar las características de ambos grupos, combinando estabilidad con potencial de retorno.